# Hyperparameteroptimierung
Import der verwendeten Pakete

In [1]:
import keras
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from joblib import load
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Laden der Trainings- und Testdatensätze

In [2]:
X_train, y_train = load('../../data/trainingsdata/LGT92-23-33_trainingsdata_for_RNN')
X_train_raw, y_train_raw = load('../../data/trainingsdata/LGT92-23-33_trainingsdata_for_RNN_RAW')
X_23_test, y_23_test = load('../../data/testdata/LGT92-23_restructured_testdata_for_RNN')
X_23_test_raw, y_23_test_raw = load('../../data/testdata/LGT92-23_testdata_for_RNN_RAW')
X_33_test, y_33_test = load('../../data/testdata/LGT92-33_restructured_testdata_for_RNN')
X_33_test_raw, y_33_test_raw = load('../../data/testdata/LGT92-33_testdata_for_RNN_RAW')

scaler_y = load('../../data/scaler/train_scaler_thesis_y.joblib')

## Funktion zum Erstellen und Kompilieren eines Keras-Modells

In [3]:
window = 24
indicators = 5
horizon = 1
def build_model(n_hidden=2, n_neurons=8, learning_rate=3e-3, input_shape=(window, indicators)):
    model = Sequential()
    model.add(LSTM(input_shape=input_shape, return_sequences=True, units=n_neurons, dropout=0.2))
    for i in range(n_hidden):
            model.add(LSTM(units=n_neurons, dropout=0.2))
    model.add(Dense(units=1))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

## Erstellen eines "KerasRegressors"
Der "KerasRegressor" ermöglicht es, Keras-Modelle in das Scikit-Learn-Framework einzubinden.

In [4]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/var/folders/yq/sk49h1496dd6jjpb80f2pyph0000gn/T/ipykernel_18719/4087176523.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


## Hyperparameteroptimierung mittels Randomized Search Cross-Validation
"RandomizedSearchCV" wird genutzt, um die besten Hyperparameter für das Keras-Modell zu finden.

Hinweis: Die Hyperparameteroptimierung kann je nach Rechenleistung sehr viel Zeit in Anspruch nehmen. Die Ergebnisse der Hyperparameteroptimierung können auf verschiedenen Plattformen variieren, da Keras-Modelle mit zufälligen Gewichten initialisiert werden. Dies führt zu unterschiedlichen Startpunkten auf der Verlustfunktion und kann die Anzahl der benötigten Epochen und die Art der Verlaufskurven beeinflussen.

In [5]:
early = EarlyStopping(monitor='val_loss', patience=9)
batch_size = 32

# Hyperparameters to be examined
param_distribs = {
    'n_hidden': [0,1,2,3],
    'n_neurons': np.arange(6, 128),
    'learning_rate': reciprocal(3e-4, 3e-2),
}

# Split Trainingsdata
tscv = TimeSeriesSplit(n_splits=2)

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=tscv)
rnd_search_cv.fit(X_train, y_train, epochs=30, validation_data=(X_23_test, y_23_test), batch_size=batch_size, callbacks=[early])

2023-06-26 07:01:25.865535: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-26 07:01:25.865658: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


/Users/maximilianrupprecht/miniforge3/envs/thesis_maximilian_rupprecht/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30


2023-06-26 07:01:26.107937: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-26 07:01:26.571059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 07:01:26.681393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 07:01:27.573652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

2023-06-26 07:06:26.996122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - ETA: 0s - loss: 0.0538

2023-06-26 07:06:30.629292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 07:06:30.684222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-26 07:06:30.804469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 5s 32ms/step - loss: 0.0538 - val_loss: 0.0114
Epoch 2/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0211 - val_loss: 0.0078
Epoch 3/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0202 - val_loss: 0.0159
Epoch 4/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0174 - val_loss: 0.0019
Epoch 5/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0174 - val_loss: 0.0054
Epoch 6/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0172 - val_loss: 0.0012
Epoch 7/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0166 - val_loss: 0.0111
Epoch 8/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0164 - val_loss: 0.0182
Epoch 9/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0149 - val_loss: 0.0438
Epoch 10/30
125/125 [==============================] - 3s 22ms/step - loss: 0.0149 - val_loss: 0.0028


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x29d847f10>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x29d8474f0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([  6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,...
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127])})

## Ergebnisse der Hyperparameteroptimierung

In [6]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'learning_rate': 0.002600495445170466, 'n_hidden': 1, 'n_neurons': 66}
-0.022613931447267532
